In [22]:
%pip install scipy
from math import ceil
from scipy.optimize import minimize

Note: you may need to restart the kernel to use updated packages.


Declaracion de variables

In [23]:
# DATOS GENERALES
total_campaña = 25000 # Kg
total_aceite = 18000 # L

# PERSONAL
capacidad_recogida = 495 # Kg/persona-dia
sueldo = 120 # €/persona-dia
alojamiento = 96 # €/dia-habitacion 
manutencion = 54 # €/persona-dia

# GASTOS FIJOS
gastos_fijos = 2600 # €/dia

# INSTALACIONES DEL TRUJAL
capacidad_prensa = 1200 # Kg/dia
capacidad_embotelladora = 1920 # L/dia

# EXTRAS
num_prensas = 1
num_embotelladoras = 1
max_prensas = 5
max_embotelladoras = 6
prensa_adicional = 8000 # €/campaña
embotelladora_adicional = 9600 # €/campaña
personas_maquina = 2
max_personas = 10

Funcion a optimizar

In [24]:
def opt_func(x: list[int]) -> tuple[float, float]: # (coste, plazo)
    personas = int(x[0])
    prensas = int(x[1])
    embotelladoras = int(x[2])

    # RECOGIDA
    dias_recogida = ceil(total_campaña / (personas * capacidad_recogida))

    # PRENSADO
    dias_prensado = ceil(total_campaña / ((1 + prensas) * capacidad_prensa))

    # EMBOTELLADO
    dias_embotellado = ceil(total_aceite / ((1 + embotelladoras) * capacidad_embotelladora))

    plazo = dias_recogida + dias_prensado + dias_embotellado
    coste_maquinas = prensas * prensa_adicional + embotelladoras * embotelladora_adicional
    gasto_personal =  plazo * (sueldo * personas + ceil(personas / 2) * alojamiento + manutencion * personas)
    coste = gasto_personal + coste_maquinas + gastos_fijos * plazo

    return (coste, plazo)


Limitaciones

In [25]:
# Constraints
def const1(x: list[int]) -> float:
    return max_personas - x[0]

def const2(x: list[int]) -> float:
    return max_prensas - (x[1] + 1) 

def const3(x: list[int]) -> float:
    return max_embotelladoras - (x[2] + 1)

def const4(x: list[int]) -> float:
    if x[1] + 1 == max_prensas and x[0] >= (x[1] + 1) * personas_maquina:
        return 0
    if x[0] > ((x[1] + 1) * personas_maquina) + (personas_maquina - 1) or x[0] < (x[1] + 1) * personas_maquina:
        return -1
    
    return 0

def const5(x: list[int]) -> float:
    if x[2] + 1 == max_embotelladoras and x[0] >= (x[2] + 1) * personas_maquina:
        return 0
    if x[0] > ((x[2] + 1) * personas_maquina) + (personas_maquina - 1) or x[0] < (x[2] + 1) * personas_maquina:
        return -1
    
    return 0

con1: dict = {'type': 'ineq', 'fun': const1}
con2: dict = {'type': 'ineq', 'fun': const2}
con3: dict = {'type': 'ineq', 'fun': const3}
con4: dict = {'type': 'eq', 'fun': const4}
con5: dict = {'type': 'eq', 'fun': const5}

cons: list[dict] = [con1, con2, con3, con4, con5]

Optimizacion y resultados

In [26]:
def check_consts(consts: list[dict], x) -> bool:
    for const in consts:
        if const['type'] == 'eq':
            if const['fun'](x) != 0:
                return False
        elif const['type'] == 'ineq':
            if const['fun'](x) < 0:
                return False
    return True

In [27]:
# Busqueda mediante fuerza bruta

min_coste = float("inf")
plazo = 0
personas = 0
prensas = 0
embotelladoras = 0
data = []
for i in range(1, max_personas + 1):
    for j in range(0, max_prensas + 1):
        for k in range(0, max_embotelladoras + 1):
            x = [i, j , k]
            if check_consts(cons, x):
                (c, p) = opt_func(x)
                c_l = total_aceite / c
                data.append([i, j, k, c, p, c_l])
                if c < min_coste or (c == min_coste and p < plazo):
                    min_coste = c
                    plazo = p
                    personas = i
                    prensas = j
                    embotelladoras = k
            

print("Se relizará en " + str(plazo) + " dias costando " + str(min_coste) + " €")
print("Se contratarán " + str(personas) + " personas y se utilizarán " + str(prensas) + " prensas extra y " + str(embotelladoras) + " embotelladoras extra")


Se relizará en 20 dias costando 113840 €
Se contratarán 6 personas y se utilizarán 2 prensas extra y 2 embotelladoras extra


In [28]:
%pip install plotly pandas nbformat jinja2
import plotly.express as px
import pandas as pd


Note: you may need to restart the kernel to use updated packages.


In [29]:
df = pd.DataFrame(data, columns=["Personas", "Prensas", "Embotelladoras", "Coste", "Plazo", "Coste/L"])
df

,Personas,Prensas,Embotelladoras,Coste,Plazo,Coste/L
0,2,0,0,173508,57,0.103742
1,3,0,0,159072,48,0.113156
2,4,1,1,118752,29,0.151576
3,5,1,1,119066,27,0.151177
4,6,2,2,113840,20,0.158117
5,7,2,2,115038,19,0.156470
6,8,3,3,122816,16,0.146561
7,9,3,3,122490,15,0.146951
8,10,4,4,133060,13,0.135277


In [30]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & Personas & Prensas & Embotelladoras & Coste & Plazo & Coste/L \\
\midrule
0 & 2 & 0 & 0 & 173508 & 57 & 0.103742 \\
1 & 3 & 0 & 0 & 159072 & 48 & 0.113156 \\
2 & 4 & 1 & 1 & 118752 & 29 & 0.151576 \\
3 & 5 & 1 & 1 & 119066 & 27 & 0.151177 \\
4 & 6 & 2 & 2 & 113840 & 20 & 0.158117 \\
5 & 7 & 2 & 2 & 115038 & 19 & 0.156470 \\
6 & 8 & 3 & 3 & 122816 & 16 & 0.146561 \\
7 & 9 & 3 & 3 & 122490 & 15 & 0.146951 \\
8 & 10 & 4 & 4 & 133060 & 13 & 0.135277 \\
\bottomrule
\end{tabular}



In [31]:
fig = px.scatter_3d(df, x='Personas', y='Prensas', z='Embotelladoras', color='Coste')
fig.show()

In [32]:
fig2 = px.line(df, x='Personas', y='Coste/L')
fig2.show()
